개념

In [ ]:
#DBSCAN(Density-Based Spatial Clustering of Applications with Noise)
#데이터가 위치하고 있는 공간 밀집도를 기준으로 클러스터를 구분

#코어 포인트
#경계 포인트
#노이즈

#클러스터 

In [ ]:
import pandas as pd
import folium

데이터 준비 및 기본 설정

In [ ]:
file_path = "2016_middle_shcool_graduates_report.xlsx"
df = pd.read_excel(file_path, header = 0)

pd.set_option("display.width", None) #출력 화면의 너비
pd.set_option("display.max_rows", 100) #출력할 행의 개수
pd.set_option("display.max_columns", 10) #출력할 열의 개수
pd.set_option("display.max_colwidth", 20) #출력할 열의 너비
pd.set_option("display.unicode.east_asian_width", True) #유니코드 사용 너비 조정

In [ ]:
df.columns.values #열 이름 배열 출력

array(['Unnamed: 0', '지역', '학교명', '코드', '유형', '주야', '남학생수', '여학생수', '일반고',
       '특성화고', '과학고', '외고_국제고', '예고_체고', '마이스터고', '자사고', '자공고', '기타진학',
       '취업', '미상', '위도', '경도'], dtype=object)

데이터 탐색

In [ ]:
df.head()

,Unnamed: 0,지역,학교명,코드,유형,...,기타진학,취업,미상,위도,경도
0,0,성북구,서울대학교사범대학부설중학교...,3,국립,...,0.004,0,0.000,37.594942,127.038909
1,1,종로구,서울대학교사범대학부설여자중학교...,3,국립,...,0.031,0,0.000,37.577473,127.003857
2,2,강남구,개원중학교,3,공립,...,0.009,0,0.003,37.491637,127.071744
3,3,강남구,개포중학교,3,공립,...,0.019,0,0.000,37.480439,127.062201
4,4,서초구,경원중학교,3,공립,...,0.010,0,0.000,37.510750,127.008900


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415 entries, 0 to 414
Data columns (total 21 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  415 non-null    int64  
 1   지역          415 non-null    object 
 2   학교명         415 non-null    object 
 3   코드          415 non-null    int64  
 4   유형          415 non-null    object 
 5   주야          415 non-null    object 
 6   남학생수        415 non-null    int64  
 7   여학생수        415 non-null    int64  
 8   일반고         415 non-null    float64
 9   특성화고        415 non-null    float64
 10  과학고         415 non-null    float64
 11  외고_국제고      415 non-null    float64
 12  예고_체고       415 non-null    float64
 13  마이스터고       415 non-null    float64
 14  자사고         415 non-null    float64
 15  자공고         415 non-null    float64
 16  기타진학        415 non-null    float64
 17  취업          415 non-null    int64  
 18  미상          415 non-null    float64
 19  위도          415 non-null    f

In [ ]:
df.describe()

,Unnamed: 0,코드,남학생수,여학생수,일반고,...,기타진학,취업,미상,위도,경도
count,415.000000,415.000000,415.000000,415.000000,415.000000,...,415.000000,415.0,415.000000,415.000000,415.000000
mean,207.000000,3.197590,126.532530,116.173494,0.623080,...,0.069571,0.0,0.001670,37.491969,127.032792
std,119.944432,0.804272,79.217906,76.833082,0.211093,...,0.235630,0.0,0.003697,0.348926,0.265245
min,0.000000,3.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,34.979940,126.639561
25%,103.500000,3.000000,80.000000,71.500000,0.566500,...,0.000000,0.0,0.000000,37.501934,126.921758
50%,207.000000,3.000000,129.000000,118.000000,0.681000,...,0.007000,0.0,0.000000,37.547702,127.013579
75%,310.500000,3.000000,177.500000,161.500000,0.758000,...,0.015000,0.0,0.003000,37.590670,127.071265
max,414.000000,9.000000,337.000000,422.000000,0.908000,...,1.000000,0.0,0.036000,37.694777,129.106974


In [ ]:
#지도에 위치 표시
mschool_map = folium.Map(location = [37.55, 126.98], tiles = "Stamen Terrain", zoom_start = 12)

In [ ]:
#중학교 위치 정보를 CircleMarker로 표시
for name, lat, lng in zip(df.학교명, df.위도, df.경도):
  folium.CircleMarker([lat, lng], radius = 5, color = "brown", fill = True, fill_color = "coral", fill_opacity = 0.7, popup = name).add_to(mschool_map)

In [ ]:
mschool_map.save("./seoul_mschool_location.html")

데이터 전처리

In [ ]:
#원핫인코딩(더미 변수)
from sklearn import preprocessing

label_encoder = preprocessing.LabelEncoder()
onehot_encoder = preprocessing.OneHotEncoder()

onehot_location = label_encoder.fit_transform(df["지역"])
onehot_code = label_encoder.fit_transform(df["코드"])
onehot_type = label_encoder.fit_transform(df["유형"])
onehot_day = label_encoder.fit_transform(df["주야"])

df["location"] = onehot_location
df["code"] = onehot_code
df["type"] = onehot_type
df["day"] = onehot_day

df.head()

,Unnamed: 0,지역,학교명,코드,유형,...,경도,location,code,type,day
0,0,성북구,서울대학교사범대학부설중학교...,3,국립,...,127.038909,16,0,1,0
1,1,종로구,서울대학교사범대학부설여자중학교...,3,국립,...,127.003857,22,0,1,0
2,2,강남구,개원중학교,3,공립,...,127.071744,0,0,0,0
3,3,강남구,개포중학교,3,공립,...,127.062201,0,0,0,0
4,4,서초구,경원중학교,3,공립,...,127.008900,14,0,0,0


모형 학습 및 검증

In [ ]:
from sklearn import cluster

#분석에 사용할 속성 선택(과학고, 외고국제고, 자사고 진학률)
columns_list = [9, 10, 13]
X = df.iloc[:, columns_list]

In [ ]:
X = preprocessing.StandardScaler().fit(X).transform(X) #정규화

In [ ]:
dbm = cluster.DBSCAN(eps = 0.2, min_samples = 5) #DBSCAN 모형 객체 생성

In [ ]:
dbm.fit(X) #모형 학습

DBSCAN(algorithm='auto', eps=0.2, leaf_size=30, metric='euclidean',
       metric_params=None, min_samples=5, n_jobs=None, p=None)

In [ ]:
cluster_label = dbm.labels_ #예측
df["Cluster"] = cluster_label #예측 결과를 데이터프레임에 추가

In [ ]:
#클러스터 값으로 그룹화하고 그룹별로 내용을 출력
grouped_cols = [0, 1, 3] + columns_list
grouped = df.groupby("Cluster")
for key, group in grouped:
  print("* key: ", key)
  print("* number: ", len(group))
  print(group.iloc[:, grouped_cols].head())
  print()

* key:  -1
* number:  234
   Unnamed: 0    지역  코드  특성화고  과학고  마이스터고
0           0  성북구     3     0.148   0.018       0.011
2           2  강남구     3     0.047   0.009       0.006
3           3  강남구     3     0.097   0.013       0.019
6           6  강남구     3     0.015   0.015       0.000
7           7  강남구     3     0.000   0.032       0.000

* key:  0
* number:  70
    Unnamed: 0    지역  코드  특성화고  과학고  마이스터고
1            1  종로구     3     0.199     0.0         0.0
13          13  서초구     3     0.032     0.0         0.0
23          23  강남구     3     0.025     0.0         0.0
28          28  서초구     3     0.040     0.0         0.0
29          29  강남구     3     0.051     0.0         0.0

* key:  1
* number:  7
    Unnamed: 0    지역  코드  특성화고  과학고  마이스터고
4            4  서초구     3     0.017   0.007         0.0
24          24  강남구     3     0.026   0.007         0.0
27          27  강남구     3     0.006   0.006         0.0
53          53  강동구     3     0.020   0.007         0.0
68          68  송파

In [ ]:
#그래프로 표현 - 시각화
colors = {
    -1 : "gray", 0 : "coral", 1 : "blue", 2 : "green", 3 : "red", 4: "purple",
    5 : "orange", 6 : "brown", 7 : "brick", 8 : "yellow", 9 : "magenta", 10 : "cyan"
}

cluster_map = folium.Map(location = [37.55, 126.98], tiles = "Stamen Terrain", zoom_start = 12)

for name, lat, lng, clus in zip(df.학교명, df.위도, df.경도, df.Cluster):
  folium.CircleMarker([lat, lng], radius = 5, color = colors[clus], fill = True, fill_color = colors[clus], fill_opacity = 0.7, popup = name).add_to(cluster_map)

cluster_map.save("./seoul_mschool_cluster.html")

KeyError: ignored